# 05 - Exportar Modelo para Producción

**Materia:** Redes Neuronales Profundas — UTN FRM

**Objetivo:** Exportar el modelo fine-tuneado para desplegarlo en la aplicación web de Streamlit.

---

## Formatos de Exportación

1. **`modelo.pth`:** Solo el state_dict (pesos). Formato liviano.
2. **`model_files/`:** Carpeta con todo lo necesario para `from_pretrained()`:
   - `model.safetensors`: Pesos en formato seguro.
   - `config.json`: Configuración del modelo.
   - `tokenizer.json` y `tokenizer_config.json`: Vocabulario y configuración del tokenizer.

## 1. Importación de Librerías

In [ ]:
import os
import torch
from transformers import BertForSequenceClassification, BertTokenizer

## 2. Configuración

In [ ]:
MODEL_DIR = "../data/model_save/"
PROD_DIR = "../prod/"

## 3. Carga y Exportación del Modelo

In [ ]:
print(f"Cargando modelo desde {MODEL_DIR}...")
model = BertForSequenceClassification.from_pretrained(MODEL_DIR)
tokenizer = BertTokenizer.from_pretrained(MODEL_DIR)
os.makedirs(PROD_DIR, exist_ok=True)

In [ ]:
# Guardar state_dict como .pth (liviano para producción)
pth_path = os.path.join(PROD_DIR, "modelo.pth")
torch.save(model.state_dict(), pth_path)
file_size = os.path.getsize(pth_path) / (1024 * 1024)
print(f"Modelo guardado: {pth_path} ({file_size:.1f} MB)")

In [ ]:
# Guardar modelo completo con save_pretrained
model_prod_dir = os.path.join(PROD_DIR, "model_files")
os.makedirs(model_prod_dir, exist_ok=True)
model.save_pretrained(model_prod_dir)
tokenizer.save_pretrained(model_prod_dir)
print(f"Modelo y tokenizer en: {model_prod_dir}")

## 4. Verificación

Verificamos que el modelo exportado se puede cargar correctamente.

In [ ]:
print("Verificando carga...")
loaded_model = BertForSequenceClassification.from_pretrained(model_prod_dir)
loaded_tokenizer = BertTokenizer.from_pretrained(model_prod_dir)
print(f"Modelo: {type(loaded_model).__name__}")
print(f"Tokenizer vocab: {loaded_tokenizer.vocab_size}")

print(f"\nArchivos en {model_prod_dir}:")
for f in os.listdir(model_prod_dir):
    size = os.path.getsize(os.path.join(model_prod_dir, f)) / (1024*1024)
    print(f"  {f} ({size:.2f} MB)")

In [ ]:
# Prueba rápida
test_text = "This game is fantastic, highly recommend!"
encoded = loaded_tokenizer(test_text, add_special_tokens=True, max_length=128,
                           padding='max_length', truncation=True, return_tensors='pt')

loaded_model.eval()
with torch.no_grad():
    result = loaded_model(**encoded)

probs = torch.softmax(result.logits, dim=1)
pred = torch.argmax(probs, dim=1).item()
confidence = probs[0][pred].item()
label = 'POSITIVA' if pred == 1 else 'NEGATIVA'

print(f"\nPrueba: '{test_text}'")
print(f"Predicción: {label} (confianza: {confidence:.2%})")
print(f"\nExportación completada. Modelo listo para producción.")

## Resumen

1. Cargamos el modelo fine-tuneado.
2. Exportamos como `modelo.pth` (state_dict).
3. Exportamos modelo completo en `model_files/` con `save_pretrained()`.
4. Verificamos carga correcta y realizamos prueba.

**El modelo está listo para la aplicación web de Streamlit (`prod/app.py`).**